In [8]:
!python3 -m pip install --upgrade pip
!apt install autodock-vina -y
!pip install plip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autodock-vina is already the newest version (1.2.3-2).
0 upgraded, 0 newly installed, 0 to remove and 131 not upgraded.
  Using cached plip-2.4.0.tar.gz (84 kB)
  Preparing metadata (setup.py) ... done
  Using cached lxml-6.0.1-cp311-cp311-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
Using cached lxml-6.0.1-cp311-cp311-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (5.2 MB)
  DEPRECATION: Building 'plip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'plip'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for plip: filename=plip-2.4.0-py3-no

In [22]:
!vina --receptor HSA_Target/4L9K/4l9k.pdbqt \
      --ligand Ligand_Pubchem_Data/Cyclochlorotine/3D/Cyclochlorotine3D.pdbqt \
      --config dockprep/4l9k-CC-Site1.txt \
      --exhaustiveness=32 --out results/4l9k_cc_site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [31]:
!vina --receptor HSA_Target/4L9K/4l9k.pdbqt \
      --ligand Ligand_Pubchem_Data/Cyclochlorotine/3D/Cyclochlorotine3D.pdbqt \
      --config dockprep/4l9k-CC-Site1.txt \
      --exhaustiveness=32 --out results/4l9k_cc_site1.pdbqt

AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [15]:
!plip -f Results/4l9k-CC-Site1/complex_4l9k_cc_site1_1.pdb -o results/4l9k-CC-Site1/

None
2025-08-26 11:46:37,713 [INFO] [plipcmd.py:134] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.4.0
2025-08-26 11:46:37,713 [INFO] [plipcmd.py:135] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2025-08-26 11:46:37,714 [INFO] [plipcmd.py:136] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2025-08-26 11:46:37,715 [INFO] [plipcmd.py:60] plip.plipcmd: starting analysis of complex_4l9k_cc_site1_1.pdb
2025-08-26 11:46:38,672 [INFO] [plipcmd.py:175] plip.plipcmd: finished analysis, find the result files in results/4l9k-CC-Site1/


In [13]:
# 1) strip Vina headers & MODEL blocks from ligand
!grep -v -E '^(MODEL|ENDMDL|REMARK|COMPND|AUTHOR|TITLE)' Results/4l9k-CC-Site1/4l9k_cc_site1_1.pdb > ligand_nometa.pdb

# 2) format ligand lines into HETATM with resname LIG, chain Z, resSeq 999 (simple awk)
awk 'BEGIN{n=1}
  /^ATOM/ || /^HETATM/ {
    # extract atom name (cols might vary), coords taken roughly from columns or tokens
    # this awk expects a space-separated Vina style line; should be robust for most vina outputs
    atomname=$3
    x=$(NF-6); y=$(NF-5); z=$(NF-4)   # adjust indexing if your pdbqt layout differs
    printf("HETATM%5d %-4s LIG Z%4d    %8.3f%8.3f%8.3f  1.00  0.00          %2s\n", n, atomname, 999, x+0, y+0, z+0, substr($0, length($0)-1)) 
    n++
  }' ligand_nometa.pdb > ligand_clean.pdb

# 3) remove headers from receptor (keep only ATOM lines)
grep -v -E '^(MODEL|ENDMDL|REMARK|COMPND|AUTHOR|TITLE)' HSA_Target/4L9K/Prep_4l9k.pdb > receptor_nometa.pdb
grep '^ATOM' receptor_nometa.pdb > receptor_atoms.pdb

# 4) concat receptor atoms + ligand HETATM
cat receptor_atoms.pdb ligand_clean.pdb > complex_for_plip.pdb
echo "END" >> complex_for_plip.pdb


IndentationError: unexpected indent (3114694230.py, line 6)